## Coroutine
 - 정의 : 각 루틴이 종속적인 관계가 아닌 대등한 관계로서, 서로를 순차적으로 호출하게끔 되어있는 함수
     - `x = yield y` 형태로 생성
     - 상호 호출은 단일 쓰레드 안에서 가능
     
     
 - 특징 : 흐름제어, 병렬처리(concurrency)를 가능하게 함
     - `yield y` 를 만난 시점에서 현 루틴은 멈추고, 다른 루틴을 진행하다가 next를 통해, 멈춰있던 루틴이 멈췄던 그 순간부터 다시 실행됨.
     - send() 함수를 통해 밖에서 coroutine 함수 안에 x 값 전달
     - next() 함수를 통해 coroutine 함수 밖으로 y값 반환 
     -  순차 실행이 아닌 병렬 처리

In [3]:
# Coroutine 상태 확인
# getgeneratorstate

from inspect import getgeneratorstate
from typing import Coroutine, final

def coroutine1(a):
    print('>>> coroutine started : {}'.format(a))
    y = yield a
    # y : 밖 루트에서 send로 받을 값
    # x : 밖 루트에 전달 할 값
    print('>>> Coroutine received : {}'.format(y))
    z = yield a+y
    print('>>>coroutine received again : {}'.format(z))
    
c3 = coroutine1(10)
print('EX1-1 -', getgeneratorstate(c3))
print(next(c3))
print('EX1-2 -', getgeneratorstate(c3))
c3.send(20)
print('EX1-3 -', getgeneratorstate(c3))
# c3.send(50)
#stop iteration error 발생.

EX1-1 - GEN_CREATED
>>> coroutine started : 10
10
EX1-2 - GEN_SUSPENDED
>>> Coroutine received : 20
EX1-3 - GEN_SUSPENDED


In [11]:
# 데코레이터 패턴으로 만들기

from functools import wraps 

def coroutine(func):
    '''Decorator runs until meets yield'''
    @wraps(func) # 설명문 독스트링 등을 싸서 같이가져가는 데코레이터
    def primer(*args, **kwargs):
        gen = func()
        # next로 gen 시동 걸어줌
        next(gen)           # total 0이 출력되지 않는 이유는 print(next(gen))이 아니고 그냥 next(gen) 이기 때문
        return gen
    return primer
        
@coroutine                   # next를 안해줘도 되게 하는 Decorator
def sumer():
    total = 0
    term = 0
    while True :              # Coroutine은 보통 무한 루프로 활용함. 여러번 사용의 장점을 살리는 것.
        term = yield total
        print('term recieved')
        total += term

su = sumer()
print('EX2-1 -',su)
print(getgeneratorstate(su))
print('EX2-2 -',su.send(100))
print('EX2-3 -',su.send(100))

EX2-1 - <generator object sumer at 0x000002D108549900>
GEN_SUSPENDED
term recieved
EX2-2 - 100
term recieved
EX2-3 - 200


In [4]:
# Coroutine 예외처리 (throw(), close())

class SampleException(Exception):
    '''설명에 사용할 예외 유형'''

def coroutine_except():
    print('>>coroutine started.')
    try:
        while True:
            try:
                x = yield 
            except SampleException:
                print('-> SampleException handled. Continuing..')
            else:   #except가 실행되지 않았을 때 실행됨
                print('-> coroutine received : {}'.format(x))
    finally: #무조건 실행되는 절.
        print('-> coroutine ending') 

exe_co = coroutine_except()

print('Ex3-1 -', next(exe_co)) 
print('EX3-2 -', exe_co.send(10))
print('EX3-3 -', exe_co.send(20))
print('EX3-4 -',exe_co.throw(SampleException)) 
print('EX3-5 -',exe_co.send(1000))
print('EX3-4 -',exe_co.close()) #GEN_CLOSED. 코루틴을 끝냈다.|

>>coroutine started.
Ex3-1 - None
-> coroutine received : 10
EX3-2 - None
-> coroutine received : 20
EX3-3 - None
-> SampleException handled. Continuing..
EX3-4 - None
-> coroutine received : 1000
EX3-5 - None
-> coroutine ending
EX3-4 - None


In [ ]:
# Coroutine Return값 확안
# e.value 를 통해

def averager_re():
    total = 0.0
    cnt = 0
    avg = None
    while True :
        term = yield
        if term is None:
            break
        total += term
        cnt += 1
        avg = total / cnt
    return 'Average : {}'.format(avg)

avger2 = averager_re()

next(avger2)
avger2.send(10)
avger2.send(20)
avger2.send(30)

# avger2.send(None) # 그냥 stop 하면 generator 끝날 때 Stopiteration error  나오니깐. 예외처리함
try :
    avger2.send(None)
except StopIteration as e :  # 예외 처리 없이 Error 발생해도 return 값 보여줌. 
    print('EX4-1 -', e.value) # coroutine의 return 값은 예외처리.value로 확인할 수 있다.

In [10]:
# yield from
# generator
# yield from 뒤의 arg를 순차적으로 반환

def gen1():
    for x in 'AB':
        yield x         #X 가 A ,B 다 끝나야 y로 넘어감
    for y in range(1,4):
        yield y

t1 = gen1()
print('EX5-1 -', list(t1))
print()

def gen2():
    yield from 'AB'     # for 문 대신 yield from 로 작성 가능
    yield from range(1,4) # yield from 뒤의 것들을 순차적으로 반환

t2 = gen2()
print('Ex6-1 -', next(t2))
print('Ex6-1 -', next(t2))
print('Ex6-1 -', next(t2))
print('EX6-2 -',list(t2))

EX5-1 - ['A', 'B', 1, 2, 3]

Ex6-1 - A
Ex6-1 - B
Ex6-1 - 1
EX6-2 - [2, 3]


In [9]:
# Coroutine 끼리 상호 제어
# yield from 이용

def gen3_sub(): #코루틴
    print(
        'sub coroutine.'
    )
    x = yield 10
    print('recv1 : ', str(x))
    x = yield 100
    print('recv2 : ', str(x))

def gen4_main(): #제너레이터
    yield from gen3_sub()
    
t5 = gen4_main()
print('EX7-1 -', next(t5))
print('EX7-2 -',t5.send(7))

sub coroutine.
EX7-1 - 10
recv1 :  7
EX7-2 - 100


In [11]:
#  y = yield from 코루틴함수()
# 코루틴함수()의 return 값을 반환한다
# 즉 코루틴 안의 코루틴에 값을 전달한다.

def accumulate():
    total = 0
    while True:
        x = (yield)         # 코루틴 바깥에서 값을 받아옴
        if x is None:       # 받아온 값이 None이면
            return total    # 합계 total을 반환
        total += x

def sum_coroutine():
    while True:
        total = yield from accumulate()    # accumulate의 반환값을 가져옴 즉, send를 통해 하위 coroutine 까지 값을 전달.
        print(total)
 
co = sum_coroutine()
next(co)
for i in range(1, 11):    # 1부터 10까지 반복
    co.send(i)            # sum_coroutine의 yield from을 통해 코루틴 accumulate에 숫자를 보냄
co.send(None)   

55
